In [ ]:
# Imports
import os
from PIL import Image, ImageOps, ImageEnhance, ImageFilter
import random
import numpy as np

In [ ]:
# Variables
input_folder = "./images"
output_folder = "./augmented_images"

rotate_angle = (1,270)
scale_range = (0.75,1.25)
max_translation=(20, 20)
brightness_range=(0.75, 1.25)
contrast_range=(0.75, 1.25)
color_range=(0.75, 1.25)
mean = 50
sigma = 25
blur_radius_range = (1, 5)


images_by_label = {}

In [104]:
# Load the original dataset

# Traverse the folder
for class_name in os.listdir(input_folder):
    class_path = os.path.join(input_folder, class_name)
    if not os.path.isdir(class_path):
        continue  # Skip if not a directory

    # Initialize list for the class if not already present
    if class_name not in images_by_label:
        images_by_label[class_name] = []

    # Traverse images in the class folder
    for file_name in os.listdir(class_path):
        if file_name.endswith(('.png', '.jpg', '.jpeg')):  # Check for valid image extensions
            file_path = os.path.join(class_path, file_name)

            # Load the image
            image = Image.open(file_path)

            # Append the image to the corresponding label's list
            images_by_label[class_name].append(image)

# Print summary
for label, images in images_by_label.items():
    print(f"Loaded {len(images)} images for label '{label}'.")


Loaded 40 images for label 'Paper'.
Loaded 54 images for label 'Rock'.
Loaded 39 images for label 'Scissor'.


In [105]:
# Create the output folder
os.makedirs(output_folder, exist_ok=True)

In [80]:
# Flip images
for label, images in images_by_label.items():
    # Create a subdirectory for the label in the output folder
    label_output_folder = os.path.join(output_folder, label)
    os.makedirs(label_output_folder, exist_ok=True)

    for idx, image in enumerate(images):
        flip_type = random.choice(["horizontal", "vertical"])
        flipped_image = None
        if flip_type == "horizontal":
            flipped_image = ImageOps.mirror(image)   
        if flip_type == "vertical":
            flipped_image = ImageOps.flip(image)

        output_file_name = f"aug_flip_{idx}.jpg"
        output_file_path = os.path.join(label_output_folder, output_file_name) 
        flipped_image.save(output_file_path)
        
print(f"Randomly flipped images have been saved in '{output_folder}'.")        

Randomly flipped images have been saved in './augmented_images'.


In [ ]:
# Rotate images
for label, images in images_by_label.items():
    # Create a subdirectory for the label in the output folder
    label_output_folder = os.path.join(output_folder, label)
    os.makedirs(label_output_folder, exist_ok=True)

    for idx, image in enumerate(images):
        angle = random.randint(*rotate_angle)
        rotated_image = image.rotate(angle, expand=True)

        output_file_name = f"aug_rotate_{idx}.jpg"
        output_file_path = os.path.join(label_output_folder, output_file_name)
        rotated_image.save(output_file_path)

print(f"Randomly rotated images have been saved in '{output_folder}'.")

Randomly rotated images have been saved in './augmented_images'.


In [90]:
# Scale images
for label, images in images_by_label.items():
    # Create a subdirectory for the label in the output folder
    label_output_folder = os.path.join(output_folder, label)
    os.makedirs(label_output_folder, exist_ok=True)

    for idx, image in enumerate(images):
        scale_factor = random.uniform(*scale_range)
        original_width, original_height = image.size
        new_width = int(original_width * scale_factor)
        new_heigth = int(original_height * scale_factor)

        scaled_image = image.resize((new_width, new_heigth), Image.Resampling.LANCZOS)

        # Center the scaled image in a canvas of the original size
        scaled_image = ImageOps.fit(scaled_image, (original_width, original_height), method=Image.Resampling.LANCZOS, centering=(0.5, 0.5))

        output_file_name = f"aug_scale_{idx}.jpg"
        output_file_path = os.path.join(label_output_folder, output_file_name)
        scaled_image.save(output_file_path)

print(f"Randomly scaled images have been saved in '{output_folder}'.")

Randomly scaled images have been saved in './augmented_images'.


In [ ]:
# Translate images (x or y direction)
for label, images in images_by_label.items():
    # Create a subdirectory for the label in the output folder
    label_output_folder = os.path.join(output_folder, label)
    os.makedirs(label_output_folder, exist_ok=True)

    for idx, image in enumerate(images):
        max_x, max_y = max_translation
        tx = random.randint(-max_x, max_x)  # Random translation along x-axis
        ty = random.randint(-max_y, max_y)  # Random translation along y-axis

        # Create a translation matrix
        translation_matrix = (1, 0, tx, 0, 1, ty)

        # Apply the transformation
        translated_image = image.transform(
            image.size, Image.AFFINE, translation_matrix, resample=Image.Resampling.BICUBIC
        )

        output_file_name = f"aug_transl_{idx}.jpg"
        output_file_path = os.path.join(label_output_folder, output_file_name)
        translated_image.save(output_file_path)

print(f"Randomly translated images have been saved in '{output_folder}'.")

Randomly translated images have been saved in './augmented_images'.


In [91]:
# Color jittering
for label, images in images_by_label.items():
    # Create a subdirectory for the label in the output folder
    label_output_folder = os.path.join(output_folder, label)
    os.makedirs(label_output_folder, exist_ok=True)

    # Augment and save each image
    for idx, image in enumerate(images):
        # Apply color jittering
        brightness_factor = random.uniform(*brightness_range)
        contrast_factor = random.uniform(*contrast_range)
        color_factor = random.uniform(*color_range)

        # Apply augmentations to the image
        color_jittered_image = ImageEnhance.Brightness(image).enhance(brightness_factor)
        color_jittered_image = ImageEnhance.Contrast(color_jittered_image).enhance(contrast_factor)
        color_jittered_image = ImageEnhance.Color(color_jittered_image).enhance(color_factor)

        # Save the augmented image
        output_file_name = f"aug_cjit_{idx}.jpg"
        output_file_path = os.path.join(label_output_folder, output_file_name)
        color_jittered_image.save(output_file_path)

print(f"Color-jittered images have been saved in '{output_folder}'.")

KeyboardInterrupt: 

In [106]:
# Adding noise
for label, images in images_by_label.items():
    # Create a subdirectory for the label in the output folder
    label_output_folder = os.path.join(output_folder, label)
    os.makedirs(label_output_folder, exist_ok=True)

    for idx, image in enumerate(images):
         # Convert image to NumPy array
        image_array = np.array(image).astype(np.float32)

        # Generate Gaussian noise
        noise = np.random.normal(mean, sigma, image_array.shape)

        # Add the noise and clip values to the valid range [0, 255]
        noisy_image = np.clip(image_array + noise, 0, 255).astype(np.uint8)

        noisy_image = Image.fromarray(noisy_image)

        output_file_name = f"aug_noise_{idx}.jpg"
        output_file_path = os.path.join(label_output_folder, output_file_name)
        noisy_image.save(output_file_path)

print(f"Noisy images have been saved in '{output_folder}'.")

KeyboardInterrupt: 

In [ ]:
# Gaussian blur
# Process each label and its images
for label, images in images_by_label.items():
    # Create a subdirectory for the label in the output folder
    label_output_folder = os.path.join(output_folder, label)
    os.makedirs(label_output_folder, exist_ok=True)

    for idx, image in enumerate(images):
        # Randomly choose a blur radius
        blur_radius = random.uniform(*blur_radius_range)

        # Apply Gaussian blur
        blurred_image = image.filter(ImageFilter.GaussianBlur(blur_radius))

        # Save the augmented image
        output_file_name = f"aug_gblur_{idx}.jpg"
        output_file_path = os.path.join(label_output_folder, output_file_name)
        blurred_image.save(output_file_path)

        # Log for debugging
        print(f"Applied Gaussian blur to image {idx} in label '{label}' with radius={blur_radius}")

print(f"Blurred images have been saved in '{output_folder}'.")
